In [4]:
import matplotlib.image as mpimg
import pandas as pd
import os
import torch
import torchaudio
from torchaudio import transforms
from torchaudio.utils import download_asset
import random
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import init
import pickle
import numpy as np
from keras.preprocessing import image
import tensorflow as tf
from tensorflow import Graph
import json
import cv2

In [5]:
class imgdata(Dataset):
    def __init__(self, df, data_path, transform = None):
        self.df = df
        self.transform = transform
        self.data_path = data_path
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        # Absolute file path of the audio file - concatenate the audio directory with
        # the relative path
        img_file = self.df.loc[idx, 'Path']
        # Get the Class ID
        class_id = self.df.loc[idx, 'ID']
        img = mpimg.imread(img_file)
        if self.transform is not None:
            img = self.transform(img)
        return img, class_id
    
    
class imgClassifier (nn.Module):
    # ----------------------------
    # Build the model architecture
    # ----------------------------
    def __init__(self):
        super().__init__()
        conv_layers = []

        # First Convolution Block with Relu and Batch Norm. Use Kaiming Initialization
        self.conv1 = nn.Conv2d(3, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.1)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        # Second Convolution Block
        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv2.weight, a=0.1)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        # Second Convolution Block
        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv3.weight, a=0.1)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        # Second Convolution Block
        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv4.weight, a=0.1)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        # Linear Classifier
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=64, out_features=450)

        # Wrap the Convolutional Blocks
        self.conv = nn.Sequential(*conv_layers)
 
    # ----------------------------
    # Forward pass computations
    # ----------------------------
    def forward(self, x):
        # Run the convolutional blocks
        x = self.conv(x)

        # Adaptive pool and flatten for input to linear layer
        x = self.ap(x)
        x = x.view(x.shape[0], -1)

        # Linear layer
        x = self.lin(x)

        # Final output
        return x
    
   
    def preprocess(model, path_to_image, path_to_dummy):
        
        MEAN = np.array([0.485, 0.456, 0.406])
        STD = np.array([0.229, 0.224, 0.225])
        
        IMG_path = [path_to_image, path_to_dummy]
        df_test = pd.DataFrame( {"Path":IMG_path, "ID":[0, 1]} )
        
        test_transform = transforms.Compose([transforms.ToPILImage(),
                                      transforms.ToTensor(),
                                      transforms.Normalize(MEAN,STD), transforms.Resize((224, 224))])  

        testDS = imgdata(df_test, IMG_path, test_transform)
        test_dl = torch.utils.data.DataLoader(testDS, batch_size=64, shuffle=False)
        
        return test_dl
    
    def predict(model, preprocessed_image):
        
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        
        correct_prediction = 0
        total_prediction = 0

        # Disable gradient updates
        with torch.no_grad():
            for data in preprocessed_image:
                # Get the input features and target labels, and put them on the GPU
                inputs, labels = data[0].to(device), data[1].to(device)

                # Normalize the inputs
                inputs_m, inputs_s = inputs.mean(), inputs.std()
                inputs = (inputs - inputs_m) / inputs_s
                
                outputs = model(inputs)
                # print(outputs)

                    # Get the predicted class with the highest score
                _, prediction = torch.max(outputs,1)
                    # Count of predictions that matched the target label
                return int(prediction[0])

In [8]:
from joblib import load

model_img = load('C:\\Users\\Tee\Desktop\\venv_Django\\SE_Project_\\SE_Site\\models_ML\\model_128_0.05', 'rb')

img_used = imgClassifier.preprocess(model_img, "D:\\Downloads\\foto2.jpg", "D:\\Downloads\\181269802.jpg")

result = imgClassifier.predict(model_img, img_used)

print(result)


276


In [29]:
alist = [0, 1, 20]

x = [i for i in alist]

print(x)

[0, 1, 20]
